In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, Bidirectional, SpatialDropout1D
from imblearn.over_sampling import SMOTE
import re
from googletrans import Translator 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.decomposition import LatentDirichletAllocation


# Load NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize Translator
translator = Translator()

# Load the original dataset
df_original = pd.read_csv('ReviewWithSentimentFlipkart.csv')

# Keep 'Summary' and 'Stance' columns
df = df_original[['Summary', 'Stance']]

# Drop rows where any of the required columns are NaN
df = df.dropna(subset=['Summary', 'Stance'])

# Ensure all entries in 'Summary' are strings
df['Summary'] = df['Summary'].astype(str)

# Print value counts of 'Stance' column
print(df['Stance'].value_counts())

# Text Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatize and remove stopwords
    return text

df['Summary'] = df['Summary'].apply(preprocess_text)

# Define back-translation function
def back_translate(text, src_language='en', mid_language='fr'):
    try:
        translated_text = translator.translate(text, src=src_language, dest=mid_language).text
        back_translated_text = translator.translate(translated_text, src=mid_language, dest=src_language).text
        return back_translated_text
    except Exception as e:
        print(f"Error during back-translation: {e}")
        return text

# Define function to augment minority class
def augment_minority_class(df, class_label, src_language='en', mid_language='fr'):
    minority_texts = df[df['Stance'] == class_label]['Summary'].tolist()
    augmented_texts = [back_translate(text, src_language, mid_language) for text in minority_texts]
    augmented_labels = [class_label] * len(augmented_texts)
    return pd.DataFrame({'Summary': augmented_texts, 'Stance': augmented_labels})

# Define threshold for minority class
threshold = 100  # Adjust based on your needs

# Identify minority classes
class_counts = df['Stance'].value_counts()
minority_classes = class_counts[class_counts < threshold].index.tolist()

# Apply back-translation for all minority classes
augmented_df_list = [augment_minority_class(df, cls) for cls in minority_classes]
df_augmented = pd.concat([df] + augmented_df_list, ignore_index=True)


# Vectorize text data for LDA
count_vectorizer_lda = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
text_data_vectorized_lda = count_vectorizer_lda.fit_transform(df_augmented['Summary'])

# Fit LDA model
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(text_data_vectorized_lda)

# Transform data into topic distributions
topic_distributions_lda = lda_model.transform(text_data_vectorized_lda)

# Add topic distributions to the dataframe
for i in range(10):  # Adjust based on the number of topics (n_components) in LDA
    df_augmented[f'topic_{i}_lda'] = topic_distributions_lda[:, i]

# Text Vectorization for ML models
tfidf = TfidfVectorizer(max_df=0.7)
X = tfidf.fit_transform(df_augmented['Summary'])
y = df_augmented['Stance']

# Handle class imbalance using SMOTE for traditional ML models
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Split dataset for ML models
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Train and evaluate ML models
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear', probability=True),  # Set probability=True for SVM
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}


# Confidence threshold
confidence_threshold = 0.8

def trust_based_accuracy(y_true, y_pred, confidence_scores, threshold):
    mask = confidence_scores >= threshold
    y_true_trust = y_true[mask]
    y_pred_trust = y_pred[mask]
    if len(y_true_trust) == 0:
        return None  # No predictions above the threshold
    return accuracy_score(y_true_trust, y_pred_trust)

for name, model in models.items():
    model.fit(X_train_ml, y_train_ml)
    y_pred = model.predict(X_test_ml)
    y_probs = model.predict_proba(X_test_ml)
    confidence_scores = y_probs.max(axis=1)
    
    # Calculate standard accuracy
    accuracy = accuracy_score(y_test_ml, y_pred)
    report = classification_report(y_test_ml, y_pred)
    print(f"{name} Results:")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)
    
    # Calculate trust-based accuracy
    trust_accuracy = trust_based_accuracy(y_test_ml, y_pred, confidence_scores, confidence_threshold)
    if trust_accuracy is not None:
        print(f"Trust-Based Accuracy for {name}: {trust_accuracy}")
    else:
        print(f"No predictions above the confidence threshold for {name}")


# Tokenizer and padding for deep learning models
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_augmented['Summary'])
X_seq_dl_lda = tokenizer.texts_to_sequences(df_augmented['Summary'])
X_pad_dl_lda = pad_sequences(X_seq_dl_lda, maxlen=100)
y_encoded = pd.get_dummies(df_augmented['Stance']).values

# Combine TF-IDF and LDA topics for deep learning models
X_combined_dl_lda = np.hstack((X_pad_dl_lda, topic_distributions_lda))

# Split dataset for deep learning models
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_combined_dl_lda, y_encoded, test_size=0.2, random_state=42)


# Define and train CNN model with TF-IDF + LDA topics
cnn_model = Sequential([
    Embedding(5000, 128, input_length=100),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')
])
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

cnn_y_pred = cnn_model.predict(X_test_dl)
cnn_y_pred_labels = cnn_y_pred.argmax(axis=1)
y_test_dl_labels = y_test_dl.argmax(axis=1)
cnn_accuracy = accuracy_score(y_test_dl_labels, cnn_y_pred_labels)
cnn_report = classification_report(y_test_dl_labels, cnn_y_pred_labels)
print("CNN Results:")
print(f"Accuracy: {cnn_accuracy}")
print("Classification Report:")
print(cnn_report)

# Calculate confidence scores for CNN
cnn_confidence_scores = cnn_y_pred.max(axis=1)

# Calculate trust-based accuracy for CNN
cnn_trust_accuracy = trust_based_accuracy(y_test_dl_labels, cnn_y_pred_labels, cnn_confidence_scores, confidence_threshold)
if cnn_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for CNN: {cnn_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for CNN")


# Define and train LSTM model with TF-IDF + LDA topics
lstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(y_encoded.shape[1], activation='softmax')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

lstm_y_pred = lstm_model.predict(X_test_dl)
lstm_y_pred_labels = lstm_y_pred.argmax(axis=1)
lstm_accuracy = accuracy_score(y_test_dl_labels, lstm_y_pred_labels)
lstm_report = classification_report(y_test_dl_labels, lstm_y_pred_labels)
print("LSTM Results:")
print(f"Accuracy: {lstm_accuracy}")
print("Classification Report:")
print(lstm_report)

# Calculate confidence scores for LSTM
lstm_confidence_scores = lstm_y_pred.max(axis=1)

# Calculate trust-based accuracy for LSTM
lstm_trust_accuracy = trust_based_accuracy(y_test_dl_labels, lstm_y_pred_labels, lstm_confidence_scores, confidence_threshold)
if lstm_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for LSTM: {lstm_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for LSTM")


# Define and train BiLSTM model with TF-IDF + LDA topics
bilstm_model = Sequential([
    Embedding(5000, 128, input_length=100),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(y_encoded.shape[1], activation='softmax')
])
bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bilstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_dl), verbose=2)

bilstm_y_pred = bilstm_model.predict(X_test_dl)
bilstm_y_pred_labels = bilstm_y_pred.argmax(axis=1)
bilstm_accuracy = accuracy_score(y_test_dl_labels, bilstm_y_pred_labels)
bilstm_report = classification_report(y_test_dl_labels, bilstm_y_pred_labels)
print("BiLSTM Results:")
print(f"Accuracy: {bilstm_accuracy}")
print("Classification Report:")
print(bilstm_report)

# Calculate confidence scores for BiLSTM
bilstm_confidence_scores = bilstm_y_pred.max(axis=1)

# Calculate trust-based accuracy for BiLSTM
bilstm_trust_accuracy = trust_based_accuracy(y_test_dl_labels, bilstm_y_pred_labels, bilstm_confidence_scores, confidence_threshold)
if bilstm_trust_accuracy is not None:
    print(f"Trust-Based Accuracy for BiLSTM: {bilstm_trust_accuracy}")
else:
    print(f"No predictions above the confidence threshold for BiLSTM")


# Compare all models
accuracy_scores_all = {
    'Naive Bayes': accuracy_score(y_test_ml, models['Naive Bayes'].predict(X_test_ml)),
    'SVM': accuracy_score(y_test_ml, models['SVM'].predict(X_test_ml)),
    'Random Forest': accuracy_score(y_test_ml, models['Random Forest'].predict(X_test_ml)),
    'CNN': cnn_accuracy,
    'LSTM': lstm_accuracy,
    'BiLSTM': bilstm_accuracy
}

best_model = max(accuracy_scores_all, key=accuracy_scores_all.get)
print(f"\nThe best model based on accuracy is: {best_model} with accuracy {accuracy_scores_all[best_model]}")

trust_accuracy_scores_all = {
    'Naive Bayes': trust_based_accuracy(y_test_ml, models['Naive Bayes'].predict(X_test_ml), models['Naive Bayes'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'SVM': trust_based_accuracy(y_test_ml, models['SVM'].predict(X_test_ml), models['SVM'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'Random Forest': trust_based_accuracy(y_test_ml, models['Random Forest'].predict(X_test_ml), models['Random Forest'].predict_proba(X_test_ml).max(axis=1), confidence_threshold),
    'CNN': cnn_trust_accuracy,
    'LSTM': lstm_trust_accuracy,
    'BiLSTM': bilstm_trust_accuracy
}

best_trust_model = max(trust_accuracy_scores_all, key=lambda k: trust_accuracy_scores_all[k] if trust_accuracy_scores_all[k] is not None else -1)
print(f"\nThe best model based on trust-based accuracy is: {best_trust_model} with trust-based accuracy {trust_accuracy_scores_all[best_trust_model]}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Moneykicks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Moneykicks\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Stance
neutral       10234
supportive    10234
oppose        10234
Name: count, dtype: int64
Naive Bayes Results:
Accuracy: 0.7664875427454811
Classification Report:
              precision    recall  f1-score   support

     neutral       0.81      0.64      0.72      2076
      oppose       0.73      0.79      0.76      2038
  supportive       0.77      0.87      0.81      2027

    accuracy                           0.77      6141
   macro avg       0.77      0.77      0.76      6141
weighted avg       0.77      0.77      0.76      6141

Trust-Based Accuracy for Naive Bayes: 0.9551656920077972
SVM Results:
Accuracy: 0.7736524995929002
Classification Report:
              precision    recall  f1-score   support

     neutral       0.74      0.74      0.74      2076
      oppose       0.78      0.73      0.76      2038
  supportive       0.80      0.84      0.82      2027

    accuracy                           0.77      6141
   macro avg       0.77      0.77      0.77      6141
weigh

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
384/384 - 10s - 27ms/step - accuracy: 0.7525 - loss: 0.6072 - val_accuracy: 0.7929 - val_loss: 0.5189
Epoch 2/5
384/384 - 10s - 25ms/step - accuracy: 0.8291 - loss: 0.4446 - val_accuracy: 0.7912 - val_loss: 0.5243
Epoch 3/5
384/384 - 9s - 23ms/step - accuracy: 0.8734 - loss: 0.3502 - val_accuracy: 0.7867 - val_loss: 0.5651
Epoch 4/5
384/384 - 9s - 24ms/step - accuracy: 0.8985 - loss: 0.2858 - val_accuracy: 0.7759 - val_loss: 0.6415
Epoch 5/5
384/384 - 9s - 24ms/step - accuracy: 0.9093 - loss: 0.2536 - val_accuracy: 0.7776 - val_loss: 0.6732
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
CNN Results:
Accuracy: 0.7775606578733105
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.72      0.74      2076
           1       0.80      0.74      0.77      2038
           2       0.78      0.87      0.82      2027

    accuracy                           0.78      6141
   macro avg       0.78      0.78      0.78      6141
weighted av

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


384/384 - 30s - 78ms/step - accuracy: 0.6894 - loss: 0.7220 - val_accuracy: 0.7722 - val_loss: 0.5747
Epoch 2/5
384/384 - 28s - 72ms/step - accuracy: 0.7875 - loss: 0.5580 - val_accuracy: 0.7854 - val_loss: 0.5581
Epoch 3/5
384/384 - 27s - 71ms/step - accuracy: 0.8074 - loss: 0.5066 - val_accuracy: 0.7792 - val_loss: 0.5699
Epoch 4/5
384/384 - 27s - 71ms/step - accuracy: 0.8232 - loss: 0.4759 - val_accuracy: 0.7833 - val_loss: 0.5722
Epoch 5/5
384/384 - 27s - 71ms/step - accuracy: 0.8310 - loss: 0.4537 - val_accuracy: 0.7857 - val_loss: 0.5804
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
LSTM Results:
Accuracy: 0.7857026542908321
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.75      0.76      2076
           1       0.80      0.75      0.77      2038
           2       0.79      0.86      0.82      2027

    accuracy                           0.79      6141
   macro avg       0.79      0.79      0.79      6141
weighted avg    

c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


384/384 - 50s - 129ms/step - accuracy: 0.6816 - loss: 0.7413 - val_accuracy: 0.7709 - val_loss: 0.5849
Epoch 2/5
384/384 - 45s - 118ms/step - accuracy: 0.7866 - loss: 0.5616 - val_accuracy: 0.7794 - val_loss: 0.5670
Epoch 3/5
384/384 - 44s - 116ms/step - accuracy: 0.8111 - loss: 0.5075 - val_accuracy: 0.7761 - val_loss: 0.5736
Epoch 4/5
384/384 - 44s - 114ms/step - accuracy: 0.8196 - loss: 0.4849 - val_accuracy: 0.7831 - val_loss: 0.5628
Epoch 5/5
384/384 - 47s - 121ms/step - accuracy: 0.8347 - loss: 0.4531 - val_accuracy: 0.7764 - val_loss: 0.5746
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step
BiLSTM Results:
Accuracy: 0.7764207783748576
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.69      0.74      2076
           1       0.79      0.74      0.76      2038
           2       0.75      0.90      0.82      2027

    accuracy                           0.78      6141
   macro avg       0.78      0.78      0.77      6141
weighted 